# Imports

In [ ]:
%matplotlib inline
import pickle
import numpy as np
import matplotlib.pyplot as plt
import datacube
dc = datacube.Datacube(config='/g/data/u46/users/brl654/datacube/simoncube.conf')

# Functions

In [ ]:
def numpy_to_xarray(array, geobox, name=None):
    """Utility to convert ndarray to DataArray, using a datacube.model.GeoBox"""
    import xarray
    coords=[xarray.Coordinate(x, geobox.coords[x].values, attrs=dict(units=geobox.coords[x].units)) 
            for x in geobox.dims]
    return xarray.DataArray(array, coords=coords, attrs=dict(crs=geobox.crs), name=name)

def rasterfile_to_xarray(file, geobox, name=None, nodata=None, src_nodata=None):
    """Blit like"""
    import rasterio
    with rasterio.open(file) as src:
        assert src.indexes == (1,) # assume single band
        band = rasterio.band(src, 1) # do not attempt to read entire extent into memory
        array = np.empty((geobox.height, geobox.width), dtype=band.dtype)
        rasterio.warp.reproject(source=band,
                                destination=array,
                                dst_crs=geobox.crs.crs_str,
                                dst_transform=geobox.affine,
                                dst_nodata=nodata,
                                src_nodata=src_nodata)
    return numpy_to_xarray(array, geobox, name)

# Find and inspect one layer

In [ ]:
ds = dc.find_datasets(product='s1_gamma0_scene')[125] # grab metadata record concerning an example dataset/layer
ds.center_time # find what time it corresponds to

This part of the code reads in the data

In [ ]:
lon = 145.5895, 146.0649
lat = -17.53, -16.7007
x = dc.load(product='s1_gamma0_scene', lat=lat, lon=lon, time=ds.center_time, output_crs='epsg:3577', resolution=(-25,25)) # load that one layer
x

In [ ]:
x.vh[:,::2,::2].plot(vmax=0.1)

In [ ]:
(x.vh != x.vh.nodata)[:,::10,::10].plot()

# (old code) Obtain all layers 
Now revised below to run faster as this code 
Expected to take 5 to 10 minutes: - 

z = dc.load(product='s1_gamma0_scene', lat=lat, lon=lon, output_crs='epsg:3577', resolution=(-25,25))

import pickle
with open('/g/data1/w85/sj9724/xarray.pickle', 'wb') as file:
    pickle.dump(z, file, protocol=-1) # write xarray to disk

with open('/g/data1/w85/sj9724/xarray.pickle', 'rb') as file:
    z = pickle.load(file) # this only takes tens of seconds. (~6GB)

# Reset. Clear everything in memory and start again with All Layers

Note this command clears EVERYTHING, so need to reload modules

# Obtain All Layers 

In [ ]:
%reset -f

In [ ]:
#Imports - 
%matplotlib inline
import pickle
import numpy as np
import matplotlib.pyplot as plt
import datacube
dc = datacube.Datacube(config='/g/data/u46/users/brl654/datacube/simoncube.conf')

In [ ]:
#Define Functions - 
def numpy_to_xarray(array, geobox, name=None):
    """Utility to convert ndarray to DataArray, using a datacube.model.GeoBox"""
    import xarray
    coords=[xarray.Coordinate(x, geobox.coords[x].values, attrs=dict(units=geobox.coords[x].units)) 
            for x in geobox.dims]
    return xarray.DataArray(array, coords=coords, attrs=dict(crs=geobox.crs), name=name)

def rasterfile_to_xarray(file, geobox, name=None, nodata=None, src_nodata=None):
    """Blit like"""
    import rasterio
    with rasterio.open(file) as src:
        assert src.indexes == (1,) # assume single band
        band = rasterio.band(src, 1) # do not attempt to read entire extent into memory
        array = np.empty((geobox.height, geobox.width), dtype=band.dtype)
        rasterio.warp.reproject(source=band,
                                destination=array,
                                dst_crs=geobox.crs.crs_str,
                                dst_transform=geobox.affine,
                                dst_nodata=nodata,
                                src_nodata=src_nodata)
    return numpy_to_xarray(array, geobox, name)

In [ ]:
ds = dc.find_datasets(product='s1_gamma0_scene')[125] # grab metadata record concerning an example dataset/layer
ds.center_time # find what time it corresponds to

In [ ]:
lon = 145.5895, 146.0649
lat = -17.53, -16.7007
x = dc.load(product='s1_gamma0_scene', lat=lat, lon=lon, time=ds.center_time, output_crs='epsg:3577', resolution=(-25,25)) # load that one layer
x

# (new code) .... Revised section - Obtain all Layers

In [ ]:
#cache = '/g/data/u46/users/brl654/radar/larry_gradproj/xarray.pickle.2'
#cache = '/g/data1/w85/sj9724/xarray.pickle'
cache = '/g/data/u46/users/sj9724/xarray.pickle.2'
try:
    with open(cache, 'rb') as file:
        z = pickle.load(file) # this only takes tens of seconds. (6GB)
except:
    # this may be 5 to 10 mintues (seeking through half a terabyte)
    z = dc.load(product='s1_gamma0_scene', lat=lat, lon=lon, output_crs='epsg:3577', resolution=(-25,25))
    with open(cache, 'wb') as file:
        pickle.dump(z, file, protocol=-1) # save result to disk

# Count observations
Patterns in the observation count often explain flaws in downstream products.

In [ ]:

counts = (z.vh!=z.vh.nodata).sum(dim='time')
counts[::4,::4].plot(cmap='inferno')

# Average signal
A naive mean will fail to account for different numbers of observations in different areas.

In [ ]:
z.vh[:,::4,::4].mean(dim='time').plot(vmax=0.1)

This bit tells me what the 'no data' value is for this dataset

In [ ]:
#x.vh.nodata

In [ ]:
average = (z.vh[:,::4,::4].sum(dim='time') / counts)
average.plot(vmax=0.1)

# Problem 1 - Steps - Identification - Solution

The above image has a strip of "problem" data. 
The following steps were taken to identify the problem -

average[700:800,100:200].plot(vmax=0.1) # Zooming in on a section of the problem area

counts[::4,::4][700:800,100:200].plot() #Looking for all x all y every 4th step

Next -

Looking at the times where nodata - identify layers to zoom into - 

(z.vh!=z.vh.nodata)[:,::4,::4][:,700:800,100:200].sum(dim=['x','y']).plot(marker='o') 

Following 3 steps - analysis and comparison of images to identify 'problem' time steps 1a. Look at last 10 time layers - 5 columns then wrap to next 5 - identify images that show 'bleeding' between areas of data/nodata 1b.Displaying the 'bleeding' time images 2.

(z.vh!=z.vh.nodata)[:,::4,::4][-10:,700:800,100:200].plot(col='time', col_wrap=5) #1a.

(z.vh!=z.vh.nodata)[:,::4,::4][-10:,700:800,100:200].time #1b.show the time of the 'bleeding' images

(z.vh)[:,::4,::4][-10:,700:800,100:200].plot(col='time', col_wrap=5, vmax=0.1)

# Problem 1 - Identified - 
areas where the data is very close to zero but not quite zero

problem = z.sel(time='2017-04-10T19:44:05.949922000').vh[::4,::4].copy()
problem.data[problem.data==0] = np.nan
problem.plot(vmax=0.00001)
(problem < 0.0001).plot() # problem identified - areas where the data is very close to zero but not quite

# Problem 1 - Solution -
We now run the same script that produced the 'problem' image but define where there is data >0.001.  Voilà! Clean image

In [ ]:
new_counts = (z.vh>0.001).sum(dim='time')
new_average = (z.vh.sum(dim='time') / new_counts)
new_average.plot(vmax=0.1)

In [ ]:
# write out the pickled file
with open('/g/data/u46/users/sj9724/average_radar', 'wb') as file:
    radar = pickle.dump(new_average, file)

# Clear everything in memory and start over with the created files
Note this command clears EVERYTHING, so need to reload modules

In [ ]:
%reset -f

In [ ]:
#Imports - 
%matplotlib inline
import pickle
import numpy as np
import matplotlib.pyplot as plt
import datacube
import pandas as pd
dc = datacube.Datacube(config='/g/data/u46/users/brl654/datacube/simoncube.conf')
import seaborn as sns

In [ ]:
#Define Functions - 
def numpy_to_xarray(array, geobox, name=None):
    """Utility to convert ndarray to DataArray, using a datacube.model.GeoBox"""
    import xarray
    coords=[xarray.Coordinate(x, geobox.coords[x].values, attrs=dict(units=geobox.coords[x].units)) 
            for x in geobox.dims]
    return xarray.DataArray(array, coords=coords, attrs=dict(crs=geobox.crs), name=name)

def rasterfile_to_xarray(file, geobox, name=None, nodata=None, src_nodata=None):
    """Blit like"""
    import rasterio
    with rasterio.open(file) as src:
        assert src.indexes == (1,) # assume single band
        band = rasterio.band(src, 1) # do not attempt to read entire extent into memory
        array = np.empty((geobox.height, geobox.width), dtype=band.dtype)
        rasterio.warp.reproject(source=band,
                                destination=array,
                                dst_crs=geobox.crs.crs_str,
                                dst_transform=geobox.affine,
                                dst_nodata=nodata,
                                src_nodata=src_nodata)
    return numpy_to_xarray(array, geobox, name)

In [ ]:
#Bring in the averaged radar file to start analysis
import pickle
with open('/g/data/u46/users/sj9724/average_radar', 'rb') as file:
    radar = pickle.load(file) 


In [ ]:
ds = dc.find_datasets(product='s1_gamma0_scene')[125] # grab metadata record concerning an example dataset/layer
ds.center_time # find what time it corresponds to
lon = 145.5895, 146.0649
lat = -17.53, -16.7007
x = dc.load(product='s1_gamma0_scene', lat=lat, lon=lon, time=ds.center_time, output_crs='epsg:3577', resolution=(-25,25)) # load that one layer
mz = rasterfile_to_xarray('/g/data/u46/users/sj9724/Landsat_mz_terrain_Larry/mz_orig.img', x.geobox, nodata=np.nan, src_nodata=-999.5)
cats = rasterfile_to_xarray('/g/data/u46/users/sj9724/Landsat_mz_terrain_Larry/terrain.img', x.geobox)

In [ ]:
radar

In [ ]:
radar.plot(vmax=0.1)

In [ ]:
mz

In [ ]:
mz.plot()

# Now we want to plot a scatter of radar vs Mz data

In [ ]:
# Need to get the radar and cats data into a single list by 'flattening' the data using- 
flatcats = cats.stack(i = ('x','y'))
flatmz = mz.stack(i = ('x','y'))
flatradar = radar.stack(i = ('x','y'))
# Check that they are still looking good
print(flatcats)
print(flatmz) 
print(flatradar)

In [ ]:
#If they are looking good, we can now do our scatter plot
fig, ax = plt.subplots()
ax.scatter(flatradar[::100], flatmz[::100] , alpha = 0.01)


# We want to limit the xaxis scale to 0-0.10 - xlimit?
ax.set_xlim([0, 0.10])
ax.set_ylim([0.8,1.01])

#Add axes labels (radar and Mzcat)(label categ.)
ax.set_xlabel('Radar [VH]')
ax.set_ylabel('Mz,Cat')
ax.set_title('Radar Vs Mz Data: TC Larry')
plt.show()

In [ ]:
plt.scatter(flatcats.data[::10], flatmz.data[::10])
plt.xlabel('category')
plt.ylabel('mz value')
plt.show()

# Produce histograms of flat data to interrogate results of scatter plot

In [ ]:
plt.hist(flatcats, 10) # plot flattened categories - flatcat
plt.show()

In [ ]:
def nanfree(x):
    return x[~np.isnan(x)]
def clean(x):
    return x[np.isfinite(x)]


#plt.hist(subset, bins=np.linspace(0,0.25,100))
#plt.show()

fig, axes = plt.subplots(nrows=11, ncols=1, figsize=(8,16), sharex=True)
axes = axes.ravel()

for i, ax in zip(range(11), axes):
    subset = clean(flatradar.data[flatcats.data == i])
    medi = np.median(subset)
    mz_i = np.unique(clean(flatmz.data[flatcats.data == i])) # find corresponding mz value
    #print(mz_i)
    assert len(mz_i) == 1
    mz_i = mz_i[0]
    
    ax.hist(subset, bins=np.linspace(0,0.25,100))
    ax.axvline(medi, color='r')
    ax.set_title(str(i) + ' ' + str(mz_i) + ' (n=' + str(len(subset)) + ')')


fig.subplots_adjust(hspace=1)
plt.show()

In [ ]:
#Remove NaN's and inf from data before plotting histogram of flatmz
#1. Id NaN's

def nanfree(mz):
    return x[~np.isnan(mz)]
def clean(mz):
    return x[np.isfinite(mz)]
#mznan = flatmz.dropna(dim = '?')
#nanlist = np.argwhere(pd.isnull(mznan))
#nanlist
# Identify NaN's in flat mz
mznan = flatmz.dropna(dim = 'i')
nanlist = np.argwhere(pd.isnull(mznan))
nanlist

In [ ]:
#Remove infinites (inf) from data
mznan.loc[~np.isfinite(mznan)] = np.nan
mznan = mznan.dropna(dim = 'i')

In [ ]:
plt.hist(mznan, 10) # plot flattened Mz,cat - flatmz
plt.show()

In [ ]:
# Identify NaN's in flat radar
radarnan = flatradar.dropna(dim = 'i')
nanlist = np.argwhere(pd.isnull(radarnan))
nanlist

In [ ]:
#Remove infinites (inf) from data
radarnan.loc[~np.isfinite(radarnan)] = np.nan
radarnan = radarnan.dropna(dim = 'i')

In [ ]:
plt.hist(radarnan, bins=np.linspace(0,1,100)) # A histogram of our flat radar data after NaNs are removed.
plt.xlabel('radar')
plt.ylabel('count')
plt.show()

In [ ]:
sns.distplot(radarnan, hist=False)

# Count observations
Patterns in the observation count often explain flaws in downstream products.

In [ ]:
cache = '/g/data/u46/users/sj9724/xarray.pickle'
lon = 145.5895, 146.0649
lat = -17.53, -16.7007 # call for partial Larry area
try:
    with open(cache, 'rb') as file:
        z = pickle.load(file) # this only takes tens of seconds. (6GB)
except:
    # this may be 5 to 10 mintues (seeking through half a terabyte)
    z = dc.load(product='s1_gamma0_scene', lat=lat, lon=lon, output_crs='epsg:3577', resolution=(-25,25))
    with open(cache, 'wb') as file:
        pickle.dump(z, file, protocol=-1) # save result to disk

In [ ]:
z.keys()

In [ ]:
counts = (z.vh!=z.vh.nodata).sum(dim='time')
counts[::4,::4].plot(cmap='inferno')

In [ ]:
z.time

In [ ]:
#time_slice = slice(None,'2016-11-12T19:36:30.178718000')
time_slice = slice('2017-02-28','2017-03-31')
z_subset = z.sel(time=time_slice)
print(z_subset.time.values)
(z_subset.vh!=z_subset.vh.nodata).sum(dim='time')[::4,::4].plot(cmap='inferno')

In [ ]:
# Need to get the radar and cats data into a single list by 'flattening' the data using- 
flatcats = cats.stack(i = ('x','y'))
flatmz = mz.stack(i = ('x','y'))
flatradar = radar.stack(i = ('x','y'))
# Check that they are still looking good
print(flatcats)
print(flatmz) 
print(flatradar)

In [ ]:
good_subset = np.isfinite(radar) & np.isfinite(mz)
good_radar = radar.data[good_subset]
good_mz = mz.data[good_subset]
#good_both = np.vstack([good_radar, good_mz])
df = pd.DataFrame(data=dict(radar_column=good_radar, mz_column=good_mz))
mapping = {1063:'bare', 1000:'water', 949:'grassy', 898:'woodland', 806:'urban', 774:'forest'}
df['mz_label'] = df.mz_column.map(mapping)
df.head() # only show a few rows

In [ ]:
smaller = df.iloc[::10]
#len(smaller)
ax = sns.violinplot(data=smaller, x='radar_column', y='mz_label', scale="area")#, cut=0)#, bw=20)
#ax.set_xlim([0,0.1])